In [8]:
import scipy
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [9]:
df = pd.read_csv("Data.csv")
df.dtypes

Direksi Perempuan      int64
Direksi Laki-Laki      int64
Proporsi Perempuan    object
Harga penutupan        int64
Total Shares           int64
Total Utang            int64
Total Asset            int64
dtype: object

In [10]:
X = (df['Direksi Perempuan'])/(df['Direksi Perempuan'] + df['Direksi Laki-Laki'])
#X = df["Direksi Perempuan"]
nilai_pasar = df["Total Shares"] * df["Harga penutupan"]
np_plus_liability = nilai_pasar + df["Total Utang"]
#Y = np_plus_liability / df["Total Asset"]
Y = nilai_pasar / df["Total Asset"]

data = pd.DataFrame({"X":X, "Y":Y})
X_mean = np.mean(data["X"])
X_std = np.std(data["X"])
Y_mean = np.mean(data["Y"])
Y_std = np.std(data["Y"])

In [11]:
data_sample = pd.DataFrame({"X": X, "Y": Y})
data_sample.describe()

,X,Y
count,155.000000,155.000000
mean,0.191673,0.509092
std,0.153234,2.036768
min,0.000000,0.007332
25%,0.038462,0.098036
50%,0.200000,0.144649
75%,0.285714,0.283211
max,0.750000,18.006205


In [12]:
# data_sample.drop(data_sample[data_sample["X"] == 0].index, inplace=True)
# np.size(np.where(data_sample["X"] == 0))

In [13]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.2488
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.619
Time:                        00:12:14   Log-Likelihood:                -329.57
No. Observations:                 155   AIC:                             663.1
Df Residuals:                     153   BIC:                             669.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6117      0.263      2.325      0.021       0.092       1.132
X             -0.5356      1.074     -0.499      0.619      -2.657       1.586
==============================================================================
Omnibus:                      266.664   Durbin-Watson:                   0.874
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30174.443
Skew:                           8.117   Prob(JB):                         0.00
Kurtosis:                      69.398   Cond. No.                         6.79
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
bX = data_sample["X"] * result.params[0]
abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [15]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.1791
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.673
Time:                        00:12:14   Log-Likelihood:                -318.09
No. Observations:                 155   AIC:                             640.2
Df Residuals:                     153   BIC:                             646.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7201      0.244      2.947      0.004       0.237       1.203
X              0.4219      0.997      0.423      0.673      -1.548       2.392
==============================================================================
Omnibus:                      273.050   Durbin-Watson:                   0.941
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33927.573
Skew:                           8.456   Prob(JB):                         0.00
Kurtosis:                      73.479   Cond. No.                         6.79
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
XQ3,XQ1 = np.percentile(data_sample["X"], [75 ,25])
XIQR = XQ3 - XQ1 

In [17]:
lower_X = XQ1 - (1.5 * XIQR)
upper_X = XQ3 + (1.5 * XIQR)

In [18]:
data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)]

56    0.75
Name: X, dtype: float64

In [19]:
data_sample.drop(data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)].index, inplace=True)

In [20]:
YQ3,YQ1 = np.percentile(data_sample['Y'], [75 ,25])
YIQR = YQ3 - YQ1 

In [21]:
lower_Y = YQ1 - (1.5 * YIQR)
upper_Y = YQ3 + (1.5 * YIQR)

In [22]:
data_sample.drop(data_sample["Y"][(data_sample["Y"] < lower_Y) | (data_sample["Y"] > upper_Y)].index, inplace=True)

In [23]:
data_sample.describe()

,X,Y
count,137.000000,137.000000
mean,0.202136,0.164788
std,0.145108,0.107457
min,0.000000,0.007332
25%,0.100000,0.091204
50%,0.200000,0.130466
75%,0.300000,0.203765
max,0.545455,0.458297


In [24]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.600
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.208
Time:                        00:12:15   Log-Likelihood:                 112.52
No. Observations:                 137   AIC:                            -221.0
Df Residuals:                     135   BIC:                            -215.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1486      0.016      9.436      0.000       0.117       0.180
X              0.0801      0.063      1.265      0.208      -0.045       0.205
==============================================================================
Omnibus:                       24.981   Durbin-Watson:                   0.890
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               32.453
Skew:                           1.171   Prob(JB):                     8.97e-08
Kurtosis:                       3.450   Cond. No.                         7.21
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
bX = data_sample["X"] * result.params["X"]

In [26]:
# Y = a + bX + e
# e = Y - a - bX

abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [27]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.935
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.166
Time:                        00:12:15   Log-Likelihood:                 176.20
No. Observations:                 137   AIC:                            -348.4
Df Residuals:                     135   BIC:                            -342.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0712      0.010      7.200      0.000       0.052       0.091
X              0.0554      0.040      1.391      0.166      -0.023       0.134
==============================================================================
Omnibus:                       31.413   Durbin-Watson:                   1.030
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               44.022
Skew:                           1.282   Prob(JB):                     2.76e-10
Kurtosis:                       4.065   Cond. No.                         7.21
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""